In [ ]:
import json
import nibiru
from nibiru import Network, ChainClient, Msg, PrivateKey

In [ ]:
nibiru.__version__

## Create a wallet and get test tokens from faucet

To create a new wallet, generate a mnemonic key using any service or using SDK call:

In [ ]:
mnemonic, private_key = PrivateKey.generate()
print(mnemonic)
# Example OUTPUT:
# enlist satisfy inspire hobby romance caught great neither kitchen unfair cage awesome update fade object eagle sun ordinary again journey spell gown tiger spin

# Your wallet address
print(private_key.to_public_key().to_address().to_acc_bech32())

Store your mnemonic key in a safe place and use it going forward.
Use faucet to get some test tokens into your wallet: https://app.nibiru.fi/faucet

Сreate your chain client and authenticate with the mnemoniс generated

In [ ]:
mnemonic = "put your mnemonic here..."
client = ChainClient(network=Network.testnet(2))
client.authenticate(mnemonic=mnemonic)
print(client.address)

Check your bank balances. If the faucet succeded - your wallet should not be empty.

In [ ]:
print(client.query.get_bank_balances(client.address))

In [ ]:
pair = "ubtc:unusd"

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.open_position(
        sender=client.address,
        pair=pair,
        is_long=True,
        quote_asset_amount=10,
        leverage=1,
        base_asset_amount_limit=0,
    )
)
print(output)

If your TX has succeeded, you should get output something like:

`ExecuteTxResp(code=0, tx_hash='A2E9935BC06C360209FA7A9760F0CF54FA0A9DE0CAB6AF2D8687886E7A5F613F', log='[]')`

Query your TX hash to get TX details:

In [ ]:
output = client.query.tx_by_hash("A2E9935BC06C360209FA7A9760F0CF54FA0A9DE0CAB6AF2D8687886E7A5F613F")
print(output)

Or directly query the state of your position:

In [ ]:
position = client.query.perp.position(pair, client.address)
print(json.dumps(position, indent=4))

We can now query the balance of the trader address using the query we saw in the previous notebook

Documentation for msgs can be found there: https://nibiru-py.readthedocs.io/en/latest/nibiru.msg.html